In [153]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import datetime
import re
import time
import random

In [154]:
data = []

In [155]:
csv_data = pd.read_csv('total-wiki-url.csv')
# csv_data

In [ ]:
# movie = 'The way back'
# year = 2010
# # url = 'https://en.wikipedia.org/wiki/Avatar_(2009_film)'
# url = 'https://en.wikipedia.org/wiki/The_Way_Back_(2010_film)'

In [156]:

for ind, row in csv_data.iterrows():
    try:
        movie = row['Movie Title']
        year = row['year']
        url = row['url']
        if type(url) == str:
            html = urlopen(url)
            soup = BeautifulSoup(html, 'html.parser')
            tables = soup.find_all('table')
            details = {}
            for table in tables:
                budgetflag = False
                rdflag = False
                budget = ''
                clbud, clud2, money, rd2, rd = '', '', '', '', ''
                rd = ''
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all('th')
                    for cell in cells:
                        if cell.text == 'Release dates' or cell.text == 'Release date':
                            rdflag = True
                        elif cell.text == 'Budget':
                            budgetflag = True
                    
                    if budgetflag:
                        cells = row.find_all('td')
                        for cell in cells:
                            # print(cell.text)
                            budget = cell.text

                            budget = re.sub('\[+\d+\]+', '', budget)
                            clbud = budget.replace(',', '')
                            clbud2 = re.findall('[0-9]+', clbud)[-1]
                            # print(clbud)
                            if 'million' in clbud:
                                money = float(clbud2) * 10**6
                            elif 'billion' in clbud:
                                money = float(clbud2) * 10**8
                            else:
                                money = float(clbud2)

                        budgetflag = False
                    
                    if rdflag:
                        cells = row.find_all('td')
                        for cell in cells:
                            # print(cell.text)
                            rd = cell.text
                            rd2 = rd.replace('-', '')
                            rd2 = re.findall('\(+\d+\)+', rd2)[-1]
                            rd2 = str(datetime.datetime.strptime(rd2, '(%Y%m%d)').date())

                        rdflag = False
                details = {'moviename': movie, 'release_date': rd, 'budget': budget, 'year': year, 'cleaned_budget': money, 'cleaned_release_date': rd2}
                print(movie)
                break
            data.append(details)
    except Exception as e:
        print(movie, e)
        # if ind == 10:
        #     break
        # time.sleep(random.randint(0, 10))
    


Toy Story 3
Alice in Wonderland
Harry Potter and the Deathly Hallows: Part 1
Inception
Shrek Forever After
The Twilight Saga: Eclipse
Iron Man 2
Tangled
Despicable Me
How to Train Your Dragon
Clash of the Titans
The King's Speech
The Chronicles of Narnia: The Voyage of the Dawn Treader
TRON: Legacy
The Karate Kid
Black Swan
Megamind
Robin Hood
Robin Hood
The Last Airbender
Little Fockers
Resident Evil: Afterlife
Shutter Island
Salt
Sex and the City 2
The Tourist
The Expendables
Grown Ups
True Grit
Gulliver's Travels
Percy Jackson & the Olympians: The Lightning Thief
The Social Network
Valentine's Day
The Sorcerer's Apprentice
Due Date
Eat Pray Love
Yogi Bear
Paranormal Activity 2
The A-Team
Jackass 3D
The Other Guys
Unstoppable
Step Up
The Book of Eli
The Town
Date Night
The Secret World of Arrietty
Legend of the Guardians: The Owls of Ga'Hoole
The Wolfman
The Bounty Hunter
Saw: The Final Chapter
Wall Street: Money Never Sleeps
The Adjustment Bureau
Predators
A Nightmare on Elm Street


In [157]:
df = pd.DataFrame(data, columns=['moviename', 'release_date', 'budget', 'year', 'cleaned_budget', 'cleaned_release_date'])

In [158]:
df

,moviename,release_date,budget,year,cleaned_budget,cleaned_release_date
0,Toy Story 3,"\nJune 12, 2010 (2010-06-12) (Taormina Film Fe...",$200 million,2010.0,200000000.0,2010-06-18
1,Alice in Wonderland,,,2010.0,,
2,Harry Potter and the Deathly Hallows: Part 1,\n11 November 2010 (2010-11-11) (Odeon Leicest...,$250 million(shared with Part 2),2010.0,2000000.0,2010-11-19
3,Inception,"\nJuly 8, 2010 (2010-07-08) (Odeon Leicester S...",$160 million,2020.0,160000000.0,2010-07-16
4,Shrek Forever After,,,2010.0,,
...,...,...,...,...,...,...
2105,Arthur Rambo,"\nJanuary 25, 2008 (2008-01-25) (United States)\n",$47.5–50 million,2022.0,50000000.0,2008-01-25
2106,Bachchhan Paandey,\n18 March 2022 (2022-03-18)[2]\n,₹185 crore (US$24 million),2022.0,24000000.0,2022-03-18
2107,Kernagis,,,2022.0,,
2108,Stop-Zemlia,,,2022.0,,


In [159]:
df.to_csv('wiki_scrapped_data_complete.csv', index=False)

In [ ]:
df2 = pd.read_csv('movie-twitter.csv')
df2

In [ ]:
df3 = df2.merge(df, how='left', right_on='moviename', left_on='Movie Title')

In [ ]:
df3

In [ ]:
df3['Release-Date'] = df3['Release-Date'].replace('0', np.nan)
df3

In [ ]:
df3['Release-Date'] = pd.to_datetime(df3['Release-Date'])
df3['cleaned_release_date'] = pd.to_datetime(df3['cleaned_release_date'])
df3

In [ ]:
df3.loc[:, 'rd'] = df3[['Release-Date', 'cleaned_release_date']].max(axis=1)

In [ ]:
df3.to_csv('twitter_missing_data_input.csv', index=False)